<a href="https://colab.research.google.com/github/WeizmannML/course2020/blob/master/Tutorial2/Pytorch_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### <font color=blue>Pytorch allows us to use automatic differentiation & use the power of GPU </font>
#### All the code snippets in the following example are taken from https://pytorch.org/tutorials/beginner/pytorch_with_examples.html 
#### Other libraries like tensorflow or JAX allows us to do the same. 

In [0]:
from __future__ import print_function
import torch

In [2]:
torch.__version__

'1.5.0+cu101'

In [0]:
import os, sys
import numpy as np
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [0]:
# ----- for automatic detection of CPU vs GPU device ---------- #
cuda_device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu' )

#### The basic working objects are torch.Tensors 

In [0]:
a = torch.tensor([[1., -1.], [1., -1.]])

In [0]:
b = torch.tensor(np.array([[1, 2, 3], [4, 5, 6]]))

In [7]:
type(a), type(b)

(torch.Tensor, torch.Tensor)

In [8]:
x = torch.zeros(5, 3, dtype=torch.long)
print(x)

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])


In [9]:
x = x.new_ones(5, 3, dtype=torch.double, device=cuda_device)      # new_* methods take in sizes
print(x)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], device='cuda:0', dtype=torch.float64)


In [10]:
x.device ### equivalently can be done as x = x.to(cuda_device)

device(type='cuda', index=0)

In [11]:
x = torch.randn_like(x, dtype=torch.float64)    # override dtype!
print(x) 

tensor([[ 1.6352,  0.1964, -0.3800],
        [ 0.3852,  0.2728,  1.4753],
        [-0.4292,  0.3997,  0.1590],
        [ 1.2749, -1.5792, -0.2541],
        [-0.6466, -0.9019,  0.1966]], device='cuda:0', dtype=torch.float64)


In [12]:
print(x.shape), print(x.size())

torch.Size([5, 3])
torch.Size([5, 3])


(None, None)

#### Conversion to numpy arrays

In [0]:
y = x.cpu().numpy()

In [15]:
y

array([[ 1.63519641,  0.19636151, -0.38003401],
       [ 0.38515931,  0.27277402,  1.47528017],
       [-0.42920675,  0.39971346,  0.15899236],
       [ 1.27492549, -1.57923226, -0.25406337],
       [-0.64664443, -0.9019441 ,  0.19664337]])

In [16]:
y.dtype

dtype('float64')

## The automatic differentiation of tensors through autograd

In [17]:
x = x.new_ones(5, 3, dtype=torch.double, device=cuda_device)

x = torch.randn_like(x, dtype=torch.double, device=cuda_device, requires_grad=True)
print(x)

tensor([[-0.3726,  0.6719,  1.0945],
        [-2.2475,  0.4804,  0.2967],
        [ 0.9076,  1.1119,  0.3307],
        [-0.8365, -0.5879,  0.6081],
        [-0.6809,  0.4516,  0.3228]], device='cuda:0', dtype=torch.float64,
       requires_grad=True)


In [18]:
b = torch.tensor(np.array([[1, 2, 3], [4, 5, 6]]), dtype=torch.double, device=cuda_device, requires_grad=True)
print(b)

tensor([[1., 2., 3.],
        [4., 5., 6.]], device='cuda:0', dtype=torch.float64,
       requires_grad=True)


In [19]:
y = x + 2
print(y)

tensor([[ 1.6274,  2.6719,  3.0945],
        [-0.2475,  2.4804,  2.2967],
        [ 2.9076,  3.1119,  2.3307],
        [ 1.1635,  1.4121,  2.6081],
        [ 1.3191,  2.4516,  2.3228]], device='cuda:0', dtype=torch.float64,
       grad_fn=<AddBackward0>)


In [20]:
y.grad_fn

In [21]:
## -- assigning a required_grad criteria -- #
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
True


In [22]:
print(a), print(b)

tensor([[0.9263, 0.9252],
        [1.4470, 7.7178]], requires_grad=True)
tensor(63.3717, grad_fn=<SumBackward0>)


(None, None)

In [23]:
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)

tensor([684.5993, 966.2435, 875.7535], grad_fn=<MulBackward0>)


In [24]:
x

tensor([0.6686, 0.9436, 0.8552], requires_grad=True)

In [25]:
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(v)

print(x.grad)

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


### Doing a backpropagation in the numpy way with torch tensors

In [26]:

dtype = torch.float
device = cuda_device
# device = torch.device("cuda:0") # Uncomment this to run on GPU

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random input and output data
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# Randomly initialize weights
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(500):
    # Forward pass: compute predicted y
    h = x.mm(w1)
    h_relu = h.clamp(min=0) # -- the clamp filters out < 0 values
    y_pred = h_relu.mm(w2)

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum().item()
    if t % 50 == 49:
        print(t, loss)

    # Backprop to compute gradients of w1 and w2 with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h) # -- we had x.T for numpy arrays

    # Update weights using gradient descent
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

49 14719.201171875
99 644.8609008789062
149 46.995662689208984
199 4.073708534240723
249 0.38698551058769226
299 0.03881172463297844
349 0.004271837882697582
399 0.0006967298686504364
449 0.00018994123092852533
499 7.616233051521704e-05


## Using the autograd function of pytorch to compute the derivatives & do backpropagation

In [27]:
# -*- coding: utf-8 -*-
dtype = torch.float
device = cuda_device
# device = torch.device("cuda:0") # Uncomment this to run on GPU

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold input and outputs.
# Setting requires_grad=False indicates that we do not need to compute gradients
# with respect to these Tensors during the backward pass.
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# Create random Tensors for weights.
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    # Forward pass: compute predicted y using operations on Tensors; these
    # are exactly the same operations we used to compute the forward pass using
    # Tensors, but we do not need to keep references to intermediate values since
    # we are not implementing the backward pass by hand.
    y_pred = x.mm(w1).clamp(min=0).mm(w2) # --- the forward pass through two layers is happening here.

    # Compute and print loss using operations on Tensors.
    # Now loss is a Tensor of shape (1,)
    # loss.item() gets the scalar value held in the loss.
    loss = (y_pred - y).pow(2).sum()
    if t % 50 == 49:
        print(t, loss.item())

    # Use autograd to compute the backward pass. This call will compute the
    # gradient of loss with respect to all Tensors with requires_grad=True.
    # After this call w1.grad and w2.grad will be Tensors holding the gradient
    # of the loss with respect to w1 and w2 respectively.
    loss.backward()

    # Manually update weights using gradient descent. Wrap in torch.no_grad()
    # because weights have requires_grad=True, but we don't need to track this
    # in autograd.
    # An alternative way is to operate on weight.data and weight.grad.data.
    # Recall that tensor.data gives a tensor that shares the storage with
    # tensor, but doesn't track history.
    # You can also use torch.optim.SGD to achieve this.
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        # Manually zero the gradients after updating weights
        w1.grad.zero_()
        w2.grad.zero_()

49 18706.25
99 1044.6990966796875
149 99.78321838378906
199 11.367023468017578
249 1.4026386737823486
299 0.18116317689418793
349 0.024268273264169693
399 0.003602081909775734
449 0.0007400322938337922
499 0.0002270481490995735


### The PyTorch nn Modeule, contains the basic NN layers

In [28]:
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# Use the nn package to define our model as a sequence of layers. nn.Sequential
# is a Module which contains other Modules, and applies them in sequence to
# produce its output. Each Linear Module computes output from input using a
# linear function, and holds internal Tensors for its weight and bias.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

# The nn package also contains definitions of popular loss functions; in this
# case we will use Mean Squared Error (MSE) as our loss function.
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-4
for t in range(500):
    # Forward pass: compute predicted y by passing x to the model. Module objects
    # override the __call__ operator so you can call them like functions. When
    # doing so you pass a Tensor of input data to the Module and it produces
    # a Tensor of output data.
    y_pred = model(x)

    # Compute and print loss. We pass Tensors containing the predicted and true
    # values of y, and the loss function returns a Tensor containing the
    # loss.
    loss = loss_fn(y_pred, y)
    if t % 50 == 49:
        print(t, loss.item())

    # Zero the gradients before running the backward pass.
    model.zero_grad()

    # Backward pass: compute gradient of the loss with respect to all the learnable
    # parameters of the model. Internally, the parameters of each Module are stored
    # in Tensors with requires_grad=True, so this call will compute gradients for
    # all learnable parameters in the model.
    loss.backward()

    # Update the weights using gradient descent. Each parameter is a Tensor, so
    # we can access its gradients like we did before.
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

49 36.841087341308594
99 3.06765079498291
149 0.3778367340564728
199 0.05494300276041031
249 0.00927006546407938
299 0.001778186997398734
349 0.00037723430432379246
399 8.689654350746423e-05
449 2.1427822503028437e-05
499 5.600826625595801e-06


### Updating the weights through PyTorch optim 

In [29]:

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# Use the nn package to define our model and loss function.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)
loss_fn = torch.nn.MSELoss(reduction='sum')

# Use the optim package to define an Optimizer that will update the weights of
# the model for us. Here we will use Adam; the optim package contains many other
# optimization algoriths. The first argument to the Adam constructor tells the
# optimizer which Tensors it should update.
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for t in range(500):
    # Forward pass: compute predicted y by passing x to the model.
    y_pred = model(x)

    # Compute and print loss.
    loss = loss_fn(y_pred, y)
    if t % 50 == 49:
        print(t, loss.item())

    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the variables it will update (which are the learnable
    # weights of the model). This is because by default, gradients are
    # accumulated in buffers( i.e, not overwritten) whenever .backward()
    # is called. Checkout docs of torch.autograd.backward for more details.
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model
    # parameters
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()

49 213.7782440185547
99 51.5125617980957
149 7.841900825500488
199 0.7790022492408752
249 0.05822024121880531
299 0.003562858561053872
349 0.00018537286086939275
399 7.69027610658668e-06
449 2.302582373658879e-07
499 4.73958117197526e-09
